In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# we don't like warnings
# you can comment the following 2 lines if you'd like to
import warnings
warnings.filterwarnings('ignore')

sns.set()

# Graphics in SVG format are more sharp and legible
#%config InlineBackend.figure_format = 'svg'

In [2]:
#org_df = pd.read_csv('./data/analy_2018-09-30_19_246.csv.gz')
org_df = pd.read_csv('./data/analy_2018-10-28_28_246.csv.gz')
#org_df = pd.read_csv('./data/analy_2018-10-28_28_.csv.gz')

print('列参数 = {}'.format(org_df.columns.values))
org_df.head()

列参数 = ['payout' 'clickid' 'pubid' 'postback_datetime' 'sub_campid'
 'publisher_payout' 'click_ip' 'is_valid_pb' 'campid' 'subid' 'source'
 'is_fraud' 'pubpb_filter' 'fraud_note' 'date' 'geo' 'click_datetime']


,payout,clickid,pubid,postback_datetime,sub_campid,publisher_payout,click_ip,is_valid_pb,campid,subid,source,is_fraud,pubpb_filter,fraud_note,date,geo,click_datetime
0,1.26,01982898-c4e2-41d9-9455-d2d2ded85637,10397,2018-10-28T00:08:18Z,NaN,1.76,79.235.92.203,True,52552750,544537,246,0,0,ok,2018-10-28,DE,2018-10-28T00:07:22Z
1,0.64,38b61ac7-e638-4c2c-b0dc-e1c94a4abf52,10548,2018-10-28T00:14:26Z,NaN,0.64,126.218.77.50,True,58335214,795ccad65c4fc27a99e2af38484942ce,246,0,900,ok,2018-10-28,JP,2018-10-27T23:07:39Z
2,0.75,c9aef454-7fbf-487b-8690-6dba58f1cffb,10548,2018-10-28T00:23:11Z,NaN,0.75,49.97.92.120,True,57174138,faffa5e44e4304775f0af7a47fa25334,246,0,900,ok,2018-10-28,JP,2018-10-27T18:18:04Z
3,3.00,1cf74231-9ad8-4747-8477-dbcb5c9ffe85,10288,2018-10-28T00:23:50Z,NaN,3.90,66.250.143.158,True,58801908,2507,246,0,900,ok,2018-10-28,US,2018-10-28T00:19:38Z
4,1.75,0a8b12a1-fc33-43b1-9fdc-a7630732bede,10291,2018-10-28T00:28:15Z,NaN,2.71,172.58.87.13,True,58333030,20197908,246,1,800,blacklist,2018-10-28,US,2018-10-28T00:26:33Z


### 0.fraud平均值

In [3]:
org_fraud_avg = round(org_df['is_fraud'].aggregate(np.sum)/org_df['is_valid_pb'].aggregate(np.sum), ndigits=2)
print('fraud平均值 = {}'.format(org_fraud_avg))

fraud平均值 = 0.25


In [4]:
def fill_org(o_df, source=None, dates=None):
    '''
    填充 fraud_payout
    :param : 
    :param : 
    :return: 
    '''
    max_citi = 3600*24*3
    max_fraud_ctit = 45
    if source:
        df = org_df[(o_df['is_valid_pb'] == True) & (o_df['source'].isin(source)) & (o_df['date'].isin(dates))]
    else:
        df = org_df[(o_df['is_valid_pb'] == True) & (o_df['date'].isin(dates))]
    df['fraud_payout'] = df['payout'] * df['is_fraud']
    df['ctit'] = ((pd.to_datetime(df['postback_datetime']) - pd.to_datetime(df['click_datetime'])) / np.timedelta64(1, 's')).astype(int)
    df['ctit'] = df['ctit'].where(df['ctit'] <= max_citi, other=max_citi)
    df['is_ctit'] = (df['ctit'] <= max_fraud_ctit).astype(np.bool).astype(np.int0)
    return df


def get_summary_by_date(df):
    dd = df.groupby(['date'])['payout','is_fraud','is_valid_pb', 'fraud_payout', 'is_ctit'].aggregate(np.sum)
    dd['payout'] = dd['payout'].round(decimals=1)
    dd['fraud_payout'] = dd['fraud_payout'].round(decimals=1)
    dd['fraud_p'] = (dd['is_fraud']/dd['is_valid_pb']).round(decimals=2)
    dd['fraud_payout_p'] = (dd['fraud_payout']/dd['payout']).round(decimals=2)
    dd['ctit_p'] = (dd['is_ctit']/dd['is_valid_pb']).round(decimals=2)
    return dd


def get_fraud_note(df):
    return df.groupby(['fraud_note'])['payout','is_fraud','is_valid_pb', 'fraud_payout'].aggregate(np.sum)


def get_tops(df, date_len, t_pb_day=10, t_payout_day=50, group_by=['pubid']):
    total_valid_pb = t_pb_day * date_len
    total_payout = t_payout_day * date_len
    dd = df.groupby(group_by)['payout','is_fraud','is_valid_pb', 'fraud_payout', 'is_ctit'].aggregate(np.sum)
    dd = dd.query('payout >= {} or is_valid_pb >= {}'.format(total_payout, total_valid_pb))
    dd['payout'] = dd['payout'].round(decimals=1)
    dd['fraud_p'] = (dd['is_fraud']/dd['is_valid_pb']).round(decimals=2)
    dd['fraud_payout_p'] = (dd['fraud_payout']/dd['payout']).round(decimals=2)
    dd['ctit_p'] = (dd['is_ctit']/dd['is_valid_pb']).round(decimals=2)
    return dd

def get_top_datas(df, key, tops):
    return df[df[key].isin(tops)]


### 0.设置source 

In [5]:
source = [246]
date_one = ['2018-10-28']
date_multi = org_df['date'].unique()
#date_one = ['2018-10-22','2018-10-23','2018-10-24','2018-10-25','2018-10-26','2018-10-27','2018-10-28']

### 1.计算一天的数据

In [6]:
df_one = fill_org(org_df, source=source, dates=date_one)
date_len = len(date_one)
print('计算 {} 天数据'.format(date_len))
get_summary_by_date(df_one)

计算 1 天数据


,payout,is_fraud,is_valid_pb,fraud_payout,is_ctit,fraud_p,fraud_payout_p,ctit_p
date,,,,,,,,
2018-10-28,466.5,22,375.0,36.5,26,0.06,0.08,0.07


In [7]:
df_one.head()

,payout,clickid,pubid,postback_datetime,sub_campid,publisher_payout,click_ip,is_valid_pb,campid,subid,source,is_fraud,pubpb_filter,fraud_note,date,geo,click_datetime,fraud_payout,ctit,is_ctit
0,1.26,01982898-c4e2-41d9-9455-d2d2ded85637,10397,2018-10-28T00:08:18Z,NaN,1.76,79.235.92.203,True,52552750,544537,246,0,0,ok,2018-10-28,DE,2018-10-28T00:07:22Z,0.00,56,0
1,0.64,38b61ac7-e638-4c2c-b0dc-e1c94a4abf52,10548,2018-10-28T00:14:26Z,NaN,0.64,126.218.77.50,True,58335214,795ccad65c4fc27a99e2af38484942ce,246,0,900,ok,2018-10-28,JP,2018-10-27T23:07:39Z,0.00,4007,0
2,0.75,c9aef454-7fbf-487b-8690-6dba58f1cffb,10548,2018-10-28T00:23:11Z,NaN,0.75,49.97.92.120,True,57174138,faffa5e44e4304775f0af7a47fa25334,246,0,900,ok,2018-10-28,JP,2018-10-27T18:18:04Z,0.00,21907,0
3,3.00,1cf74231-9ad8-4747-8477-dbcb5c9ffe85,10288,2018-10-28T00:23:50Z,NaN,3.90,66.250.143.158,True,58801908,2507,246,0,900,ok,2018-10-28,US,2018-10-28T00:19:38Z,0.00,252,0
4,1.75,0a8b12a1-fc33-43b1-9fdc-a7630732bede,10291,2018-10-28T00:28:15Z,NaN,2.71,172.58.87.13,True,58333030,20197908,246,1,800,blacklist,2018-10-28,US,2018-10-28T00:26:33Z,1.75,102,0


In [8]:
get_fraud_note(df_one)

,payout,is_fraud,is_valid_pb,fraud_payout
fraud_note,,,,
blacklist,36.53,22,22.0,36.53
ok,429.95,0,353.0,0.00


#### 1.1 以pubid维度分析

In [9]:
top_pub_dd = get_tops(df_one, date_len, t_pb_day=10, t_payout_day=50, group_by=['pubid'])
top_pub_dd = top_pub_dd.sort_values(by=['fraud_payout_p','fraud_p'], ascending=False)
top_pub_dd

,payout,is_fraud,is_valid_pb,fraud_payout,is_ctit,fraud_p,fraud_payout_p,ctit_p
pubid,,,,,,,,
10236,58.1,6,34.0,10.40,3,0.18,0.18,0.09
10291,64.5,5,38.0,8.32,2,0.13,0.13,0.05
10397,96.7,3,72.0,5.25,16,0.04,0.05,0.22
10288,32.0,2,29.0,1.40,0,0.07,0.04,0.00
10488,49.9,1,26.0,1.51,0,0.04,0.03,0.00
10548,91.3,1,141.0,0.64,0,0.01,0.01,0.00


In [10]:
top_pub_dd.index.values

array([10236, 10291, 10397, 10288, 10488, 10548])

In [11]:
top_pub_dd = top_pub_dd.sort_values(by=['fraud_payout','fraud_p'], ascending=False)
top_pub_dd

,payout,is_fraud,is_valid_pb,fraud_payout,is_ctit,fraud_p,fraud_payout_p,ctit_p
pubid,,,,,,,,
10236,58.1,6,34.0,10.40,3,0.18,0.18,0.09
10291,64.5,5,38.0,8.32,2,0.13,0.13,0.05
10397,96.7,3,72.0,5.25,16,0.04,0.05,0.22
10488,49.9,1,26.0,1.51,0,0.04,0.03,0.00
10288,32.0,2,29.0,1.40,0,0.07,0.04,0.00
10548,91.3,1,141.0,0.64,0,0.01,0.01,0.00


In [12]:
top_pub_dd.index.values

array([10236, 10291, 10397, 10488, 10288, 10548])

#### 1.2 以pubid+subid维度分析

In [13]:
top_sub_dd = get_tops(df_one, date_len, t_pb_day=3, t_payout_day=5, group_by=['subid']).sort_values(by=['payout'], ascending=False)
top_sub_dd

,payout,is_fraud,is_valid_pb,fraud_payout,is_ctit,fraud_p,fraud_payout_p,ctit_p
subid,,,,,,,,
544537,156.5,9,107.0,15.65,19,0.08,0.10,0.18
faffa5e44e4304775f0af7a47fa25334,30.6,0,47.0,0.00,0,0.00,0.00,0.00
d8c81ae323fbe4faf5bd3e2e75deb1d8,26.6,0,42.0,0.00,0,0.00,0.00,0.00
b706d875d71d2cf3fad85b08f68601ac,26.1,1,40.0,0.64,0,0.02,0.02,0.00
90675,11.5,0,7.0,0.00,0,0.00,0.00,0.00
7_659607,9.7,0,4.0,0.00,0,0.00,0.00,0.00
2507,9.0,0,3.0,0.00,0,0.00,0.00,0.00
20150142,8.1,0,4.0,0.00,1,0.00,0.00,0.25
92701,7.8,1,5.0,1.51,0,0.20,0.19,0.00


In [14]:
top_sub_dd = top_sub_dd.sort_values(by=['fraud_payout_p','fraud_p'], ascending=False)
top_sub_dd

,payout,is_fraud,is_valid_pb,fraud_payout,is_ctit,fraud_p,fraud_payout_p,ctit_p
subid,,,,,,,,
20202162,5.8,1,2.0,4.00,0,0.50,0.69,0.00
20197908,5.3,1,3.0,1.75,1,0.33,0.33,0.33
20177884,4.8,1,3.0,1.51,0,0.33,0.31,0.00
92701,7.8,1,5.0,1.51,0,0.20,0.19,0.00
544537,156.5,9,107.0,15.65,19,0.08,0.10,0.18
b706d875d71d2cf3fad85b08f68601ac,26.1,1,40.0,0.64,0,0.02,0.02,0.00
faffa5e44e4304775f0af7a47fa25334,30.6,0,47.0,0.00,0,0.00,0.00,0.00
d8c81ae323fbe4faf5bd3e2e75deb1d8,26.6,0,42.0,0.00,0,0.00,0.00,0.00
90675,11.5,0,7.0,0.00,0,0.00,0.00,0.00


In [15]:
top_sub_dd = top_sub_dd.sort_values(by=['fraud_payout','fraud_p'], ascending=False)
top_sub_dd

,payout,is_fraud,is_valid_pb,fraud_payout,is_ctit,fraud_p,fraud_payout_p,ctit_p
subid,,,,,,,,
544537,156.5,9,107.0,15.65,19,0.08,0.10,0.18
20202162,5.8,1,2.0,4.00,0,0.50,0.69,0.00
20197908,5.3,1,3.0,1.75,1,0.33,0.33,0.33
20177884,4.8,1,3.0,1.51,0,0.33,0.31,0.00
92701,7.8,1,5.0,1.51,0,0.20,0.19,0.00
b706d875d71d2cf3fad85b08f68601ac,26.1,1,40.0,0.64,0,0.02,0.02,0.00
faffa5e44e4304775f0af7a47fa25334,30.6,0,47.0,0.00,0,0.00,0.00,0.00
d8c81ae323fbe4faf5bd3e2e75deb1d8,26.6,0,42.0,0.00,0,0.00,0.00,0.00
90675,11.5,0,7.0,0.00,0,0.00,0.00,0.00


#### 1.3 以campid+pubid+subid维度分析

In [16]:
top_camp_dd_temp = get_tops(df_one, date_len, t_pb_day=10, t_payout_day=50, group_by=['campid']).sort_values(by=['payout'], ascending=False)
top_camp_dd_temp

,payout,is_fraud,is_valid_pb,fraud_payout,is_ctit,fraud_p,fraud_payout_p,ctit_p
campid,,,,,,,,
58333030,89.2,11,51.0,19.25,0,0.22,0.22,0.00
52552750,73.1,0,58.0,0.00,17,0.00,0.00,0.29
54114288,54.4,4,36.0,6.04,2,0.11,0.11,0.06
57174138,44.2,0,59.0,0.00,0,0.00,0.00,0.00
59137467,44.0,1,11.0,4.00,0,0.09,0.09,0.00
55334293,43.2,1,24.0,1.80,1,0.04,0.04,0.04
58335212,27.8,0,48.0,0.00,0,0.00,0.00,0.00
58427059,25.5,2,15.0,3.40,6,0.13,0.13,0.40
58335214,22.4,1,35.0,0.64,0,0.03,0.03,0.00


In [17]:
top_campids = top_camp_dd_temp.index.values
top_campids

array([58333030, 52552750, 54114288, 57174138, 59137467, 55334293,
       58335212, 58427059, 58335214, 58335210])

In [18]:
top_camp_dd = get_tops(get_top_datas(df_one, 'campid', top_campids), date_len, t_pb_day=3, t_payout_day=5, group_by=['campid', 'pubid', 'subid'])
top_camp_dd.sort_values(by=['campid', 'pubid', 'subid'], ascending=False)

payout  is_fraud  \
campid   pubid subid                                                
58427059 10236 544537                              13.6         2   
58335214 10548 faffa5e44e4304775f0af7a47fa25334     5.8         0   
               d8c81ae323fbe4faf5bd3e2e75deb1d8     3.8         0   
               b706d875d71d2cf3fad85b08f68601ac     5.1         1   
58335212 10548 faffa5e44e4304775f0af7a47fa25334     8.7         0   
               d8c81ae323fbe4faf5bd3e2e75deb1d8     7.5         0   
               b706d875d71d2cf3fad85b08f68601ac     6.4         0   
         10288 84662337                             1.7         0   
58335210 10548 faffa5e44e4304775f0af7a47fa25334     2.7         0   
               d8c81ae323fbe4faf5bd3e2e75deb1d8     4.8         0   
               b706d875d71d2cf3fad85b08f68601ac     2.7         0   
58333030 10488 90675                                7.0         0   
         10397 544537                              12.2         3   
         10291 20259705                             5.2         0   
         10236 544537                              33.2         4   
57174138 10548 faffa5e44e4304775f0af7a47fa25334    13.5         0   
               d8c81ae323fbe4faf5bd3e2e75deb1d8    10.5         0   
               b706d875d71d2cf3fad85b08f68601ac    12.0         0   
         10288 10250697                             3.8         0   
55334293 10488 33524                                5.4         0   
         10236 544537                               7.2         0   
54114288 10488 92701                                6.0         1   
               90675                                4.5         0   
         10397 544537                              15.1         0   
52552750 10397 544537                              69.3         0   

                                                 is_valid_pb  fraud_payout  \
campid   pubid subid                                                         
58427059 10236 544537                                    8.0          3.40   
58335214 10548 faffa5e44e4304775f0af7a47fa25334          9.0          0.00   
               d8c81ae323fbe4faf5bd3e2e75deb1d8          6.0          0.00   
               b706d875d71d2cf3fad85b08f68601ac          8.0          0.64   
58335212 10548 faffa5e44e4304775f0af7a47fa25334         15.0          0.00   
               d8c81ae323fbe4faf5bd3e2e75deb1d8         13.0          0.00   
               b706d875d71d2cf3fad85b08f68601ac         11.0          0.00   
         10288 84662337                                  3.0          0.00   
58335210 10548 faffa5e44e4304775f0af7a47fa25334          5.0          0.00   
               d8c81ae323fbe4faf5bd3e2e75deb1d8          9.0          0.00   
               b706d875d71d2cf3fad85b08f68601ac          5.0          0.00   
58333030 10488 90675                                     4.0          0.00   
         10397 544537                                    7.0          5.25   
         10291 20259705                                  3.0          0.00   
         10236 544537                                   19.0          7.00   
57174138 10548 faffa5e44e4304775f0af7a47fa25334         18.0          0.00   
               d8c81ae323fbe4faf5bd3e2e75deb1d8         14.0          0.00   
               b706d875d71d2cf3fad85b08f68601ac         16.0          0.00   
         10288 10250697                                  5.0          0.00   
55334293 10488 33524                                     3.0          0.00   
         10236 544537                                    4.0          0.00   
54114288 10488 92701                                     4.0          1.51   
               90675                                     3.0          0.00   
         10397 544537                                   10.0          0.00   
52552750 10397 544537                                   55.0          0.00   

                                                 is_ctit  fraud_p  \
campid   

In [19]:
top_camp_dd = top_camp_dd.reset_index()

In [20]:
top_dd = top_camp_dd.sort_values(by=['fraud_payout_p','fraud_p'], ascending=False)
top_dd

,campid,pubid,subid,payout,is_fraud,is_valid_pb,fraud_payout,is_ctit,fraud_p,fraud_payout_p,ctit_p
12,58333030,10397,544537,12.2,3,7.0,5.25,0,0.43,0.43,0.00
3,54114288,10488,92701,6.0,1,4.0,1.51,0,0.25,0.25,0.00
24,58427059,10236,544537,13.6,2,8.0,3.40,2,0.25,0.25,0.25
10,58333030,10236,544537,33.2,4,19.0,7.00,0,0.21,0.21,0.00
21,58335214,10548,b706d875d71d2cf3fad85b08f68601ac,5.1,1,8.0,0.64,0,0.12,0.13,0.00
0,52552750,10397,544537,69.3,0,55.0,0.00,15,0.00,0.00,0.27
1,54114288,10397,544537,15.1,0,10.0,0.00,1,0.00,0.00,0.10
2,54114288,10488,90675,4.5,0,3.0,0.00,0,0.00,0.00,0.00
4,55334293,10236,544537,7.2,0,4.0,0.00,0,0.00,0.00,0.00
5,55334293,10488,33524,5.4,0,3.0,0.00,0,0.00,0.00,0.00


In [21]:
top_dd = top_camp_dd.sort_values(by=['fraud_payout','fraud_p'], ascending=False)
top_dd

,campid,pubid,subid,payout,is_fraud,is_valid_pb,fraud_payout,is_ctit,fraud_p,fraud_payout_p,ctit_p
10,58333030,10236,544537,33.2,4,19.0,7.00,0,0.21,0.21,0.00
12,58333030,10397,544537,12.2,3,7.0,5.25,0,0.43,0.43,0.00
24,58427059,10236,544537,13.6,2,8.0,3.40,2,0.25,0.25,0.25
3,54114288,10488,92701,6.0,1,4.0,1.51,0,0.25,0.25,0.00
21,58335214,10548,b706d875d71d2cf3fad85b08f68601ac,5.1,1,8.0,0.64,0,0.12,0.13,0.00
0,52552750,10397,544537,69.3,0,55.0,0.00,15,0.00,0.00,0.27
1,54114288,10397,544537,15.1,0,10.0,0.00,1,0.00,0.00,0.10
2,54114288,10488,90675,4.5,0,3.0,0.00,0,0.00,0.00,0.00
4,55334293,10236,544537,7.2,0,4.0,0.00,0,0.00,0.00,0.00
5,55334293,10488,33524,5.4,0,3.0,0.00,0,0.00,0.00,0.00


### 2. 计算多天的数据

In [22]:
return
df_multi = fill_org(org_df, source, dates=date_multi)
date_len = len(date_multi)
get_summary_by_date(df_multi)

SyntaxError: 'return' outside function (<ipython-input-22-0afd05dcd628>, line 1)

In [ ]:
get_fraud_note(df_multi)

#### 根据fraud_payout_p和fraud_p排序

In [ ]:
top_dd = get_tops(df_multi, date_len)
top_dd = top_dd.sort_values(by=['fraud_payout_p','fraud_p'], ascending=False)
top_dd

In [ ]:
top_dd.index.values

#### 根据fraud_payout和fraud_p排序

In [ ]:
top_dd = top_dd.sort_values(by=['fraud_payout','fraud_p'], ascending=False)
top_dd

In [ ]:
top_dd.index.values

#### 根据ctit_p和fraud_payout_p排序

In [ ]:
top_dd = top_dd.sort_values(by=['ctit_p','fraud_payout'], ascending=False)
top_dd

In [ ]:
top_dd.index.values

In [ ]:
top_df = get_top_datas(df_multi, 'pubid', top_dd.index.values)

In [ ]:
dt = pd.pivot_table(top_df, values=['payout'], index=['fraud_note'], columns=['pubid'], aggfunc='sum', margins=False,fill_value=0)
#dt = dt.reset_index()
dt

In [ ]:
sns.pairplot(top_df[['payout', 'pubid', 'is_fraud', 'is_valid_pb', 'fraud_payout', 'ctit', 'is_ctit']]);

In [ ]:

#plt.figure(figsize=(1200, 800))
#sns.set(style="ticks", color_codes=True,rc={'figure.figsize':(100, 200)})
#sns.catplot(x='pubid', y='payout', hue="is_fraud", kind="swarm", palette="Set1", data=top_df);
sns.catplot(x="pubid", y="payout", hue="is_fraud", kind="violin", inner="stick", split=True, palette="pastel", data=top_df);

In [ ]:
sns.distplot(df['payout']);

In [ ]:
dd.index.values

In [ ]:
#with pd.plotting.plot_params.use('x_compat', True):
#    for i in dt.columns.values:
#        dt[i].plot();
        #dt[i].rolling(window=7).mean().plot();     
dt.plot(subplots=True, grid=True, figsize=(20, 200), use_index=True, xticks=range(0, len(dt.index.values)));

In [ ]:
dt.rolling(window=7).mean().plot(subplots=True, grid=True, figsize=(20, 200), use_index=True, xticks=range(0, len(dt.index.values)));

In [ ]:
import matplotlib.pyplot as plt
x=dt.index
y1 = dt
y2 = dt.rolling(window=3).mean()

#plt.plot(x,y1,'r',x,y2,'h',);
dt.plot(subplots=True, grid=True, figsize=(20, 200), use_index=True, xticks=range(0, len(dt.index.values)));

In [ ]:
c_date = '2018-10-17'
total_payout = 100
fraud_low_weight = 0.8
fraud_high_weight = 1.2

dd = df[df['date']==c_date]
#dd = df

In [ ]:
fraud_avg = dd['is_fraud'].aggregate(np.sum)/dd['is_valid_pb'].size
fraud_avg

In [ ]:
dd = dd.groupby(['pubid'])['payout','is_fraud','is_valid_pb'].aggregate(np.sum)
dd['fraud_p'] = (dd['is_fraud']/dd['is_valid_pb']).round(decimals=2)

In [ ]:
dd.query('payout > {} & fraud_p >= {}'.format(total_payout, fraud_avg*fraud_high_weight)).sort_values(by=['payout'], ascending=False)

In [ ]:
dd.query('payout > {} & fraud_p <= {}'.format(total_payout, fraud_avg*fraud_low_weight)).sort_values(by=['payout'], ascending=False)

In [ ]:
sns.pairplot(data[['payout', 'is_fraud']])

In [ ]:
sns.boxplot(x='payout', y='fraud_p', data=data)